In [ ]:
import pyodbc
import pandas as pd
import numpy as np
from konlpy.tag import Okt
import re
from tqdm import tqdm
from collections import Counter

In [ ]:
# 빈도 분석



DSNNAME = "Tibero6"
DBUSER = "youngho"
DBPWD = "nare"

cnxn = pyodbc.connect('DSN='+DSNNAME+';UID='+DBUSER+';PWD='+DBPWD)
cursor = cnxn.cursor()
cursor.execute("SELECT REST_ID , REVIEW , LABEL_TF from REVIEW_COLLECT;")

row = cursor.fetchall()

cnxn.commit()
cnxn.close()

In [ ]:
row_frm = pd.DataFrame(row, columns=['REST_ID' , 'LABEL_TF' ,'REVIEW_COLLECT']

In [ ]:
row_group = row_frm.groupby('REST_ID')
group_id = list(set(row_frm.REST_ID.tolist()))

In [ ]:
group_list = []
for gp in group_id :
    gp = int(gp)
    # print(f'group_{gp}')
    globals()[f'group_{gp}'] = row_group.get_group(gp)
    group_list.append(globals()[f'group_{gp}'])

In [ ]:
wordcloud_list = []
# pbar = tqdm(group_list[:1])
pbar = tqdm(ttttlst)
# 차원 분할, 어절 추출
def pos(copy_review) :
    split_review = copy_review.REVIEW.tolist()
    # 3차원 구조, 2차원으로 변환
    chage_space = [item for sublist in split_review for item in sublist ]
    
    # 명사와 형용사만 추출
    encoded = [word for word,shape in chage_space if shape in ['Adjective' , 'Noun'] if word not in stopwords]
    # 개수 세기
    word_counts = Counter(encoded)
    #df = pd.DataFrame.from_dict(word_counts, orient='index', columns=['COUNT'])
    df = pd.DataFrame.from_records(list(word_counts.items()), columns=['WORD', 'COUNT'])
    df = df.sort_values('COUNT' , ascending=False).reset_index(drop=True)
    return df[:10]
                          
for group_once in pbar :
    
    stopwords = [ '도', '는', '다', '의', '가', '이', '은', 
                  '한', '에', '하', '고', '을', '를', '인', '듯', 
                  '과', '와', '네', '들', '듯', '지', '임', '게', 
                  '는', '이', '했', '슴', '음', '것', '거', '로',
                  '들', '거', '곳', '분', '원', '더', '왜', '해',
                  '수', '할', '그', '함', '돈', '번', '두', '개',
                  '건', '내', '저', '만', '갈', '걸', '제', '명', 
                  '분', '또', '및', '안', '른', '때', '접', '나',
                  '인데', '이가', '했었','해서', '있다', '이다', 
                  '습니다', '했는데', '입니다' ]
    
    # 정규 표현식 수행
    copy_review = group_once.copy()
    copy_review.REVIEW = copy_review.REVIEW.str.replace(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','' , regex=True)
    copy_review.dropna(subset=['REVIEW'], inplace=True)
    
    if len(copy_review) == 0 :
        no_wordcloud = pd.DataFrame({'WORD':None , 'COUNT':None ,'LABEL_ATF':None , 'REST_ID':int(copy_review.REST_ID.iloc[0])})
        wordcloud_list.append(no_wordcloud)
    else : 
        # okt pos 어절 분할
        copy_review.REVIEW = copy_review.REVIEW.apply(lambda x: okt.pos(x, norm=True, stem=True))
        copy_review.dropna(subset=['REVIEW'], inplace=True)
    
        if len(copy_review) == 0 :
            no_wordcloud = pd.DataFrame({'WORD':None , 'COUNT':None ,'LABEL_ATF':None , 'REST_ID':int(copy_review.REST_ID.iloc[0])})
            wordcloud_list.append(no_wordcloud)
        else :
            T_copy_review = copy_review.query('LABEL_TF == "T"')
            F_copy_review = copy_review.query('LABEL_TF == "F"')

            word_counts = pos(copy_review) # all
            word_counts['LABEL_ATF'] = 'A'
            T_word_counts = pos(T_copy_review) # T
            T_word_counts['LABEL_ATF'] = 'T'
            F_word_counts = pos(F_copy_review) # F
            F_word_counts['LABEL_ATF'] = 'F'

    #         print(word_counts)
    #         print(T_word_counts)
    #         print(F_word_counts)

            once_wordcloud = pd.concat([word_counts , T_word_counts , F_word_counts] , axis = 0).reset_index(drop=True)
            once_wordcloud['REST_ID'] = int(copy_review.REST_ID.iloc[0])
            wordcloud_list.append(once_wordcloud)
            # print(once_wordcloud)
    #print('------------------------------------one group end------------------------------------')
pbar.close()
wordcloud_frm = pd.concat(wordcloud_list , axis=0)